<a href="https://colab.research.google.com/github/Rukks89/music-player-streamlit/blob/main/Consent_Management_Blockchain_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install web3 py-solc-x eth-tester


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.1 MB/s eta 0:00:00


In [2]:
consent_contract = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract ConsentManager {

    struct Consent {
        string purpose;
        uint256 expiry;
        bool active;
    }

    mapping(address => mapping(string => Consent)) private consents;

    event ConsentCreated(address patient, string dataId, string purpose, uint256 expiry);
    event ConsentRevoked(address patient, string dataId);
    event AccessChecked(address requester, string dataId, bool allowed);

    function createConsent(
        string memory dataId,
        string memory purpose,
        uint256 expiry
    ) public {
        consents[msg.sender][dataId] = Consent(purpose, expiry, true);
        emit ConsentCreated(msg.sender, dataId, purpose, expiry);
    }

    function revokeConsent(string memory dataId) public {
        consents[msg.sender][dataId].active = false;
        emit ConsentRevoked(msg.sender, dataId);
    }

    function checkAccess(
        address patient,
        string memory dataId,
        string memory purpose
    ) public returns (bool) {
        Consent memory c = consents[patient][dataId];

        bool allowed = (
            c.active &&
            keccak256(bytes(c.purpose)) == keccak256(bytes(purpose)) &&
            block.timestamp <= c.expiry
        );

        emit AccessChecked(msg.sender, dataId, allowed);
        return allowed;
    }
}
"""


In [3]:
from solcx import compile_source, install_solc

install_solc("0.8.0")

compiled = compile_source(
    consent_contract,
    solc_version="0.8.0"
)

contract_id, contract_interface = compiled.popitem()
bytecode = contract_interface["bin"]
abi = contract_interface["abi"]


In [4]:
from web3 import Web3
from eth_tester import EthereumTester

w3 = Web3(Web3.EthereumTesterProvider())
w3.is_connected()


Package 'py-evm' not found.
Package 'py-evm' not found.
Package 'py-evm' not found.
Package 'py-evm' not found.


/usr/local/lib/python3.12/dist-packages/eth_tester/backends/__init__.py:45: UserWarning: Ethereum Tester: No backend was explicitly set, and no *full* backends were available.  Falling back to the `MockBackend` which does not support all EVM functionality.  Please refer to the `eth-tester` documentation for information on what backends are available and how to set them.  Your py-evm package may need to be updated.
  backend_class = get_chain_backend_class()
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


True

In [7]:
acct = w3.eth.accounts[0]

Consent = w3.eth.contract(abi=abi, bytecode=bytecode)

tx_hash = Consent.constructor().transact({
    'from': acct,
    'gas': 3000000
})

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

tx_receipt.contractAddress


In [9]:
acct = w3.eth.accounts[0]

Consent = w3.eth.contract(
    abi=abi,
    bytecode=bytecode
)

tx_hash = Consent.constructor().transact({
    'from': acct,
    'gas': 3000000
})

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# 🔴 THIS LINE IS CRITICAL
contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

contract.address


In [11]:
from solcx import compile_source, install_solc

install_solc("0.8.0")

compiled = compile_source(consent_contract, solc_version="0.8.0")
_, contract_interface = compiled.popitem()

abi = contract_interface["abi"]
bytecode = contract_interface["bin"]


In [12]:
w3.is_connected()


True

In [13]:
acct = w3.eth.accounts[0]

Consent = w3.eth.contract(
    abi=abi,
    bytecode=bytecode
)

tx_hash = Consent.constructor().transact({
    'from': acct,
    'gas': 3000000
})

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# 🔴 DO NOT MISS THIS
contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

print("Contract deployed at:", contract.address)


Contract deployed at: None


In [1]:
!pip install web3 py-solc-x eth-tester py-evm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.5 MB/s eta 0:00:00


In [2]:
from web3 import Web3
from eth_tester import EthereumTester, PyEVMBackend

eth_tester = EthereumTester(PyEVMBackend())
w3 = Web3(Web3.EthereumTesterProvider(eth_tester))

print("Connected:", w3.is_connected())
print("Accounts:", w3.eth.accounts)


Connected: True
Accounts: ['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf', '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF', '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69', '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718', '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276', '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141', '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb', '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C', '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c', '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528']


In [3]:
from solcx import compile_source, install_solc

install_solc("0.8.0")

consent_contract = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract ConsentManager {

    struct Consent {
        string purpose;
        uint256 expiry;
        bool active;
    }

    mapping(address => mapping(string => Consent)) private consents;

    event ConsentCreated(address patient, string dataId, string purpose, uint256 expiry);
    event ConsentRevoked(address patient, string dataId);
    event AccessChecked(address requester, string dataId, bool allowed);

    function createConsent(string memory dataId, string memory purpose, uint256 expiry) public {
        consents[msg.sender][dataId] = Consent(purpose, expiry, true);
        emit ConsentCreated(msg.sender, dataId, purpose, expiry);
    }

    function revokeConsent(string memory dataId) public {
        consents[msg.sender][dataId].active = false;
        emit ConsentRevoked(msg.sender, dataId);
    }

    function checkAccess(address patient, string memory dataId, string memory purpose) public returns (bool) {
        Consent memory c = consents[patient][dataId];
        bool allowed = (
            c.active &&
            keccak256(bytes(c.purpose)) == keccak256(bytes(purpose)) &&
            block.timestamp <= c.expiry
        );
        emit AccessChecked(msg.sender, dataId, allowed);
        return allowed;
    }
}
"""

compiled = compile_source(consent_contract, solc_version="0.8.0")
_, contract_interface = compiled.popitem()

abi = contract_interface["abi"]
bytecode = contract_interface["bin"]


In [4]:
acct = w3.eth.accounts[0]

Consent = w3.eth.contract(
    abi=abi,
    bytecode=bytecode
)

tx_hash = Consent.constructor().transact({
    'from': acct,
    'gas': 3000000
})

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print("Deployment receipt:", tx_receipt)

contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

print("Contract address:", contract.address)


Deployment receipt: AttributeDict({'blockHash': HexBytes('0xdead9270fb5cc088e945dc64b14209dbffbf80b50c6e0243206ecc6a2fdc554a'), 'blockNumber': 1, 'contractAddress': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b', 'cumulativeGasUsed': 610185, 'effectiveGasPrice': 1000000000, 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf', 'gasUsed': 610185, 'logs': [], 'state_root': b'\x01', 'status': 1, 'to': '', 'transactionHash': HexBytes('0x8094944161bf86d339d1029a2ed81a9044f0f2a109ef0405ee6f5f5db1052b83'), 'transactionIndex': 0, 'type': 2})
Contract address: 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b


In [5]:
import time

patient = w3.eth.accounts[0]

expiry_time = int(time.time()) + 30 * 24 * 60 * 60

tx = contract.functions.createConsent(
    "lab_report_001",
    "treatment",
    expiry_time
).transact({
    'from': patient,
    'gas': 200000
})

w3.eth.wait_for_transaction_receipt(tx)

"Consent granted successfully"


'Consent granted successfully'

In [6]:
contract.address


'0xF2E246BB76DF876Cef8b38ae84130F4F55De395b'

In [7]:
import time

patient = w3.eth.accounts[0]
doctor = w3.eth.accounts[1]

expiry_time = int(time.time()) + 30 * 24 * 60 * 60  # 30 days

tx = contract.functions.createConsent(
    "lab_report_001",
    "treatment",
    expiry_time
).transact({
    'from': patient,
    'gas': 200000
})

w3.eth.wait_for_transaction_receipt(tx)

"Consent granted successfully"


'Consent granted successfully'

In [8]:
allowed = contract.functions.checkAccess(
    patient,
    "lab_report_001",
    "treatment"
).call({'from': doctor})

allowed


True

In [9]:
tx = contract.functions.revokeConsent(
    "lab_report_001"
).transact({
    'from': patient,
    'gas': 100000
})

w3.eth.wait_for_transaction_receipt(tx)

"Consent revoked successfully"


'Consent revoked successfully'

In [10]:
allowed = contract.functions.checkAccess(
    patient,
    "lab_report_001",
    "treatment"
).call({'from': doctor})

allowed


False

In [11]:
events = contract.events.AccessChecked().get_logs()
events


[]